# Extraindo Dados do Twitter com Python


Uma aplicação para monitorar os principais resultados do Twitter e seu devido volume na rede social ao longo do tempo. O objetivo é saber as principais tendências e o que estão falando sobre o assunto.

+ [Search Tweets](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets#)
+ [Buscando tweets com Python](https://www.alura.com.br/artigos/buscando-tweets-com-python)

In [17]:
!pip install geocoder

## Importando Bibliotecas

In [18]:
import geocoder
import toml
import tweepy
import json
import pandas as pd

## Arquivo com as Chaves de Acesso ao Twitter

In [19]:
# abrir o arquivo com as credenciais de acesso
with open('config.toml') as config:
  # ler o arquivo e salvar as chaves nas variáveis
  config = toml.loads(config.read())
  APP_NAME = config['APP_NAME']
  API_KEY = config['API_KEY']
  API_KEY_SECRET = config['API_KEY_SECRET']
  ACCESS_TOKEN = config['ACCESS_TOKEN']
  ACCESS_TOKEN_SECRET = config['ACCESS_TOKEN_SECRET']

In [20]:
APP_NAME

'TwitterAoCubo'

## Autenticando no Twitter

In [21]:
# função para autenticação do usuário
auth = tweepy.OAuthHandler(API_KEY, API_KEY_SECRET)
# função para acesso ao app com os tokens
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
# autenticação na API do Twitter
api = tweepy.API(auth)

## Top Trends Twitter

In [39]:
# Verificando as localizações disponíveis para o trends
available_loc = api.trends_available()
df = pd.json_normalize(available_loc)
df.head(3)

,name,url,parentid,country,woeid,countryCode,placeType.code,placeType.name
0,Worldwide,http://where.yahooapis.com/v1/place/1,0,,1,None,19,Supername
1,Winnipeg,http://where.yahooapis.com/v1/place/2972,23424775,Canada,2972,CA,7,Town
2,Ottawa,http://where.yahooapis.com/v1/place/3369,23424775,Canada,3369,CA,7,Town


In [30]:
# Filtrando Brasil nas localizações disponíveis
df.query('country == "Brazil"')

,name,url,parentid,country,woeid,countryCode,placeType.code,placeType.name
78,Brasília,http://where.yahooapis.com/v1/place/455819,23424768,Brazil,455819,BR,7,Town
79,Belém,http://where.yahooapis.com/v1/place/455820,23424768,Brazil,455820,BR,7,Town
80,Belo Horizonte,http://where.yahooapis.com/v1/place/455821,23424768,Brazil,455821,BR,7,Town
81,Curitiba,http://where.yahooapis.com/v1/place/455822,23424768,Brazil,455822,BR,7,Town
82,Porto Alegre,http://where.yahooapis.com/v1/place/455823,23424768,Brazil,455823,BR,7,Town
83,Recife,http://where.yahooapis.com/v1/place/455824,23424768,Brazil,455824,BR,7,Town
84,Rio de Janeiro,http://where.yahooapis.com/v1/place/455825,23424768,Brazil,455825,BR,7,Town
85,Salvador,http://where.yahooapis.com/v1/place/455826,23424768,Brazil,455826,BR,7,Town
86,São Paulo,http://where.yahooapis.com/v1/place/455827,23424768,Brazil,455827,BR,7,Town
87,Campinas,http://where.yahooapis.com/v1/place/455828,23424768,Brazil,455828,BR,7,Town


In [46]:
# Definindo a localização
loc = geocoder.osm('Brazil')
print(loc)

<[OK] Osm - Geocode [Brasil]>


In [47]:
# Buscando o id da localização basedado na lat e long
closest_loc = api.trends_closest(loc.lat,loc.lng)
print(json.dumps(closest_loc,indent=1))

[
 {
  "name": "Brazil",
  "placeType": {
   "code": 12,
   "name": "Country"
  },
  "url": "http://where.yahooapis.com/v1/place/23424768",
  "parentid": 1,
  "country": "Brazil",
  "woeid": 23424768,
  "countryCode": "BR"
 }
]


In [48]:
# Montando dataset do Top trends da localização informada
trends  = api.trends_place(closest_loc[0]["woeid"])

In [49]:
# Verificando os dados das Top trends selecionadas
print('Data:',trends[0]['as_of'])
print('Data Trend:',trends[0]['created_at'])
print('Location:',trends[0]['locations'][0]['name'])

Data: 2021-12-28T12:23:17Z
Data Trend: 2021-12-26T11:40:20Z
Location: Brazil


In [50]:
# Exibindo os Top Trends Brasil
df_trends = pd.json_normalize(trends[0]['trends'])
df_trends.head()

,name,url,promoted_content,query,tweet_volume
0,Cuca,http://twitter.com/search?q=Cuca,None,Cuca,23769.0
1,Jorge Jesus,http://twitter.com/search?q=%22Jorge+Jesus%22,None,%22Jorge+Jesus%22,34505.0
2,#BolsonaroVagabundo,http://twitter.com/search?q=%23BolsonaroVagabundo,None,%23BolsonaroVagabundo,NaN
3,Atlético,http://twitter.com/search?q=Atl%C3%A9tico,None,Atl%C3%A9tico,39008.0
4,Paulo Sousa,http://twitter.com/search?q=%22Paulo+Sousa%22,None,%22Paulo+Sousa%22,34740.0


## Extraindo Dados do Twitter

In [37]:
# Realizando uma busca de dados pelo Top Trend
results = api.search(q='Cuca')

# Organizando os dados
users = []
tweets = []
for tweet in results:
  users.append(tweet.user.name)
  tweets.append(tweet.text)
  print(f'User: {tweet.user.name} | Tweet: {tweet.text}')

User: @Verainacio44 | Tweet: RT @Falagalo13: ❌ O pedido de demissão do técnico Cuca não surpreendeu a diretoria do @Atletico. A informação que chega, é que o técnico es…
User: Maria Rebelo | Tweet: Melhor de todos os tempos. Só atrás de Telê. Obrigada Cuca!!!! https://t.co/dy4ZVdG6OC
User: Gilson Luiz | Tweet: @ArnaldoJRibeiro Ja ta tudo desenhado, Jorge Jesus no Flamengo, Galo sem tecnico vai aproveitar que o Paulo Sousa e… https://t.co/uXDYvKtWcQ
User: M🅰️theus ⭐⭐ | Tweet: RT @jason13cam: 🐓 Cuca e sua história no GALO

⚔️224 jogos
✅128 vitórias
⏸48 empates 
❌48 derrotas 
📊64.2% aproveitamento
🏆Libertadores
🏆Ca…
User: renus | Tweet: Contei pro meu pai que o Cuca saiu do Galo: https://t.co/cF9RLAFdWK
User: Adauto Freitas | Tweet: @dieg0gal0 @martinsymiguel Abel Ferreira não colocou ele no bolso porra nenhuma. Você não viu o jogo. So viu o resu… https://t.co/8Y29Cks0we
User: Rodrigo Gonçalves | Tweet: O cuca não bate bem da cuca msm 🤦🏻‍♂️
User: Kaua | Tweet: RT @thirfernandes: Cuca deci

In [38]:
# Montando um dataset com os tweets extraidos do Twitter
df_tweets = pd.DataFrame()
df_tweets['users'] = users
df_tweets['tweets'] = tweets
df_tweets.head()

,users,tweets
0,@Verainacio44,RT @Falagalo13: ❌ O pedido de demissão do técn...
1,Maria Rebelo,Melhor de todos os tempos. Só atrás de Telê. O...
2,Gilson Luiz,"@ArnaldoJRibeiro Ja ta tudo desenhado, Jorge J..."
3,M🅰️theus ⭐⭐,RT @jason13cam: 🐓 Cuca e sua história no GALO\...
4,renus,Contei pro meu pai que o Cuca saiu do Galo: ht...
